In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

def build_model(input_shape, action_size):
    model = tf.keras.Sequential([
        Input(shape=input_shape),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(action_size, activation='linear')
    ])
    return model

import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from collections import deque
import random

# Set up the environment
env = gym.make("Pendulum-v1")

num_actions = 10
action_bins = np.linspace(-2.0, 2.0, num_actions)

def build_model(input_shape, action_size):
    model = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(action_size, activation='linear')
    ])
    return model

# Create the DQN model
state_shape = env.observation_space.shape
model = build_model(state_shape, num_actions)

# Hyperparameters
learning_rate = 0.001
gamma = 0.95  # Discount factor for past rewards

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')

# Function to update the Q-values
def update_q_values(minibatch):
    states = np.vstack([transition[0] for transition in minibatch])
    actions = np.array([transition[1] for transition in minibatch])
    rewards = np.array([transition[2] for transition in minibatch])
    next_states = np.vstack([transition[3] for transition in minibatch])
    dones = np.array([transition[4] for transition in minibatch])

    q_values = model.predict(states)
    q_values_next = model.predict(next_states)

    for i in range(len(minibatch)):
        if dones[i]:
            q_values[i][actions[i]] = rewards[i]
        else:
            q_values[i][actions[i]] = rewards[i] + gamma * np.max(q_values_next[i])
    return states, q_values


# Hyperparameters
epochs = 200
batch_size = 32
memory = deque(maxlen=10000)

# Function to choose an action
def choose_action(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(num_actions)
    q_values = model.predict(state.reshape(1, -1))
    return np.argmax(q_values[0])

# Function to discretize the action for the environment
def get_continuous_action(discrete_action):
    return [action_bins[discrete_action]]

# Training loop
for epoch in range(epochs):
    state = env.reset()
    done = False
    total_reward = 0 

    while not done:
        action = choose_action(state, epsilon=1.0/(epoch+1))
        continuous_action = get_continuous_action(action)
        print(env.step(continuous_action))
        next_state, reward, done, _, _ = env.step(continuous_action)

        # Handle state whether it's a tuple or directly an array
        state_array = state[0] if isinstance(state, tuple) else state
        next_state_array = next_state[0] if isinstance(next_state, tuple) else next_state
        memory.append((state_array, action, reward, next_state_array, done))

        state = next_state
        total_reward += reward
        # Train the model if memory is sufficient
        if len(memory) > batch_size:
            minibatch = random.sample(memory, batch_size)
            x_train, y_train = update_q_values(minibatch)
            model.train_on_batch(x_train, y_train)

    rewards_history.append(total_reward)
    print(f"Epoch: {epoch}, Total Reward: {total_reward}")

    if (epoch + 1) % 500 == 0:
        plot_rewards(rewards_history, epoch + 1)




1/1 [==============================] - ETA: 0s

c:\Users\zachu\.conda\envs\tensorflow_env1\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
c:\Users\zachu\.conda\envs\tensorflow_env1\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\zachu\.conda\envs\tensorflow_env1\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\zachu\.conda\envs\tensorflow_env1\lib\site-packages\gym\utils\passive_env_checker.py:252: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'numpy.ndarray'>
  logger.warn(


1/1 [==============================] - 0s 56ms/step


C:\Users\zachu\AppData\Local\Temp\ipykernel_12796\2987097267.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  target_f[0][0] = target


1/1 [==============================] - 0s 17ms/step


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 2) + inhomogeneous part.